### Import necessary spark and python stuff

In [4]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [5]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

### select the data
*path_on_hdfs_to_your_data*

In [6]:
df = spark.read.json("/user/rucio01/traces/traces.2018-04*")

### check the structure

In [7]:
# df.printSchema()

### Filter for loadAvg
_https://metricmgr.cern.ch/metric/20002/_

In [8]:
df.head()

Row(account=u'pilot', appid=None, catStart=None, clientState=u'DONE', dataset=u'', datasetName=None, datasetScope=u'', dataset_scope=None, duid=None, eventType=u'download', eventVersion=u'1.14.11', filename=u'EventService_premerge_3886111819_13607623-3886111819-13041798597-3795-10.tar', filesize=None, guid=None, hostname=u'b6ac037cc1.cern.ch', ip=None, localSite=None, name=None, pq=None, protocol=u's3', relativeStart=None, remoteSite=u'CERN-PROD_ES', scope=u'transient', stateReason=None, suspicious=None, taskid=None, timeEnd=None, timeStart=None, traceId=u'e68c1b1bbee84d5a96dcfc2f38dc26bb', traceIp=u'::ffff:188.184.146.18', traceTimeentry=u'2018-03-31T22:00:00.005599', traceTimeentryUnix=1522533600.005599, transferEnd=1522533599.975359, transferStart=1522533588.042424, url=None, usr=None, usrdn=None, uuid=u'53cf181ec38b4e2a8885b2a81109d505', validateStart=None, version=None)

### Create temporary table view

In [9]:
# body_schema = spark.read.json(df_loadAvg.rdd.map(lambda row: row.body)).schema
df.createOrReplaceTempView("traces")

### Use PySpark to query download events and split dataset DID into new features

In [10]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)

In [11]:
df_ps = spark.sql("SELECT account, clientState, dataset, eventType, eventVersion, filename, filesize, remoteSite, traceTimeentryUnix\
                               FROM traces \
                               WHERE dataset != 'None' AND dataset != '' AND dataset != '[null]' AND (dataset LIKE 'data%' OR dataset LIKE 'mc%')")\

In [12]:
#df_ps.show(n=5)

In [13]:
# df_nulls = df_ps.where(col("account").isNull())

In [14]:
# df_nulls.show(n=5)

In [15]:
# replacing null values in column account with the value of column eventVersion of the same row
# df_ps2 = df_ps.withColumn("account",coalesce(df.account,df.eventVersion))
# df_ps2.show(n=5)

df_ps = df_ps.dropna()
#df_ps.show(n=5)

In [16]:
split_col = split(df_ps['dataset'], "\.")
df_ps = df_ps.withColumn('project', split_col.getItem(0))
df_ps = df_ps.withColumn('run_number', split_col.getItem(1))
df_ps = df_ps.withColumn('stream_name', split_col.getItem(2))
df_ps = df_ps.withColumn('prod_step', split_col.getItem(3))
df_ps = df_ps.withColumn('datatype', split_col.getItem(4))
df_ps = df_ps.withColumn('dataset_version', split_col.getItem(5))
#df_ps.show(n=5)


### Perform One-Hot-encoding to make categorical data numerical (necessary for linear regression)

In [17]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer
# https://spark.apache.org/docs/2.1.0/ml-features.html#onehotencoder


In [18]:

#categoricalFeatures = ['account', 'clientState', 'dataset', 'eventType', 'eventVersion', 'filename', 'remoteSite', 'project', 'stream_name', 'prod_step', 'datatype'] # feature dataset version causes error: "Caused by: org.apache.spark.SparkException: StringIndexer encountered NULL value. To handle or skip NULLS, try setting StringIndexer.handleInvalid."

encoded = df_ps

#for elem in categoricalFeatures:
    #encoded = encoded.withColumn(elem + "Hash", hash(encoded[elem]))

#encoded.show(n=5)

In [19]:
encoded.createOrReplaceTempView("encoded")

In [20]:
# TODO: linear regression like here: https://www.datacamp.com/community/tutorials/apache-spark-tutorial-machine-learning#basics

df_ps_counts = spark.sql("SELECT dataset AS dataset_counts, count(*) as hits \
                               FROM traces \
                               WHERE dataset != 'None' AND dataset != '' AND dataset != '[null]' AND (dataset LIKE 'data%' OR dataset LIKE 'mc%')\
                               GROUP BY  dataset \
                               ORDER BY 2 DESC")
#df_ps_counts.show(n=5)

In [21]:
df_ps_counts.createOrReplaceTempView("counts")

In [22]:
df_ps_merged = spark.sql("SELECT * \
                               FROM encoded \
                               INNER JOIN counts ON encoded.dataset = counts.dataset_counts")

In [23]:
#df_ps_merged.show(n=5)

In [24]:
#df_ps_merged.count()

In [25]:
df_ps_merged.createOrReplaceTempView("mergedCheck")
test = spark.sql("SELECT dataset, dataset_counts \
                               FROM mergedCheck \
                               WHERE dataset = dataset_counts")

In [26]:
#test.show(n=5)

In [27]:
#test.count()

In [28]:
df_ps_merged = df_ps_merged.drop('dataset_counts')

In [29]:
# refresh merged view so that all columns are in it
df_ps_merged.createOrReplaceTempView("merged")

In [30]:
#df_ps_counts.count()

In [31]:
#encoded.count()

In [32]:
#df_ps_merged.count()

In [33]:
temp_class = spark.sql("SELECT dataset_counts, CASE WHEN hits > 10 THEN 1 ELSE 0 END AS cache FROM counts")\

In [34]:
#temp_class.show(n=5)

In [35]:
temp_class.createOrReplaceTempView("temp_classes")

In [36]:
df_ps_merged = spark.sql("SELECT * \
                               FROM merged \
                               JOIN temp_classes ON merged.dataset = temp_classes.dataset_counts")

In [37]:
#df_ps_merged.show(n=5)

In [38]:
#df_ps_merged.count()

In [39]:
#df_ps_merged_dropped = df_ps_merged.drop('account','clientState','dataset','eventType','eventVersion','filename','remoteSite','project','stream_name','prod_step','datatype','dataset_version', 'dataset_counts','accountIndex','clientStateIndex','datasetIndex','eventTypeIndex','eventVersionIndex','filenameIndex','remoteSiteIndex','projectIndex','stream_nameIndex','prod_stepIndex','datatypeIndex')
#df_ps_merged_dropped.show(n=5)
df_ps_merged_dropped = df_ps_merged.drop('dataset_version', 'dataset_counts')



In [40]:
df_ps_merged_dropped.printSchema()

root
 |-- account: string (nullable = true)
 |-- clientState: string (nullable = true)
 |-- dataset: string (nullable = true)
 |-- eventType: string (nullable = true)
 |-- eventVersion: string (nullable = true)
 |-- filename: string (nullable = true)
 |-- filesize: string (nullable = true)
 |-- remoteSite: string (nullable = true)
 |-- traceTimeentryUnix: double (nullable = true)
 |-- project: string (nullable = true)
 |-- run_number: string (nullable = true)
 |-- stream_name: string (nullable = true)
 |-- prod_step: string (nullable = true)
 |-- datatype: string (nullable = true)
 |-- hits: long (nullable = false)
 |-- cache: integer (nullable = false)



In [41]:
df_ps_final = df_ps_merged_dropped.withColumn("filesize_double",df_ps_merged_dropped.filesize.cast(DoubleType()))
df_ps_final = df_ps_final.withColumn("run_number_double",df_ps_merged_dropped.run_number.cast(DoubleType()))



In [42]:
df_ps_final.printSchema()

root
 |-- account: string (nullable = true)
 |-- clientState: string (nullable = true)
 |-- dataset: string (nullable = true)
 |-- eventType: string (nullable = true)
 |-- eventVersion: string (nullable = true)
 |-- filename: string (nullable = true)
 |-- filesize: string (nullable = true)
 |-- remoteSite: string (nullable = true)
 |-- traceTimeentryUnix: double (nullable = true)
 |-- project: string (nullable = true)
 |-- run_number: string (nullable = true)
 |-- stream_name: string (nullable = true)
 |-- prod_step: string (nullable = true)
 |-- datatype: string (nullable = true)
 |-- hits: long (nullable = false)
 |-- cache: integer (nullable = false)
 |-- filesize_double: double (nullable = true)
 |-- run_number_double: double (nullable = true)



In [43]:
#df_ps_final.show(n=5)

In [44]:
df_ps_final = df_ps_final.drop('run_number', 'filesize')

In [45]:
df_ps_final = df_ps_final.select("cache","account","clientState","dataset","eventVersion","remoteSite","project","prod_step","datatype")


In [46]:
#df_ps_final.show(n=5)

In [47]:
df_ps_final.printSchema()

root
 |-- cache: integer (nullable = false)
 |-- account: string (nullable = true)
 |-- clientState: string (nullable = true)
 |-- dataset: string (nullable = true)
 |-- eventVersion: string (nullable = true)
 |-- remoteSite: string (nullable = true)
 |-- project: string (nullable = true)
 |-- prod_step: string (nullable = true)
 |-- datatype: string (nullable = true)



### Scale Numeric Features

In [48]:
df_ps_final = df_ps_final.dropna()

In [49]:
#numericalFeatures = ['traceTimeentryUnix', 'accountHash', 'clientStateHash', 'datasetHash', 'eventVersionHash', 'filenameHash', 'remoteSiteHash', 'projectHash', 'stream_nameHash', 'prod_stepHash', 'datatypeHash', 'filesize_double', 'run_number_double'] # feature dataset version causes error: "Caused by: org.apache.spark.SparkException: StringIndexer encountered NULL value. To handle or skip NULLS, try setting StringIndexer.handleInvalid."

#for elem in numericalFeatures:
    #df_ps_final = df_ps_final.withColumn(elem + "Scaled", col(elem) / df_ps_final.agg(stddev_samp(elem)).first()[0])

In [50]:
#df_ps_final = df_ps_final.dropna()
df_ps_final.show(n=5)

+-----+-------+-----------+--------------------+------------+--------------------+------------+---------+------------+
|cache|account|clientState|             dataset|eventVersion|          remoteSite|     project|prod_step|    datatype|
+-----+-------+-----------+--------------------+------------+--------------------+------------+---------+------------+
|    1|  ichiu|       DONE|data15_13TeV.0028...|     1.14.11|   BNL-OSG2_DATADISK|data15_13TeV|    deriv|DAOD_HIGG4D4|
|    1|  ichiu|       DONE|data15_13TeV.0028...|     1.14.11|   BNL-OSG2_DATADISK|data15_13TeV|    deriv|DAOD_HIGG4D4|
|    1|  ichiu|       DONE|data15_13TeV.0028...|     1.14.11|RO-07-NIPNE_DATADISK|data15_13TeV|    deriv|DAOD_HIGG4D4|
|    1|  ichiu|       DONE|data15_13TeV.0028...|     1.14.11|   BNL-OSG2_DATADISK|data15_13TeV|    deriv|DAOD_HIGG4D4|
|    1|  ichiu|       DONE|data15_13TeV.0028...|     1.14.11|   BNL-OSG2_DATADISK|data15_13TeV|    deriv|DAOD_HIGG4D4|
+-----+-------+-----------+--------------------+

### Separating the features from the target variable

In [54]:
from pyspark.ml.feature import VectorIndexer

for elem in ["account","clientState","dataset","eventVersion","remoteSite","project","prod_step","datatype"]:
    indexer = VectorIndexer(inputCol=elem, outputCol= elem + "indexed", maxCategories=10)
    indexerModel = indexer.fit(df_ps_final)
    output = indexerModel.transform(df_ps_final)

output.show(n=5)

IllegalArgumentException: u'requirement failed: Column account must be of type org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7 but was actually StringType.'

In [ ]:
#output.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_ps_final.columns]).show(n=10)


In [ ]:
#output.show(n=5)

In [ ]:
output = output.select("cache", "features")

### Split the data into train and test sets

In [ ]:
output_sample = output.sample(False,0.5,1234)
#.limit(10000)
#output_sample.show(n=5)

In [ ]:
train_data, test_data = output.randomSplit([.8,.2],seed=1234)

train_data.count()

In [ ]:
test_data.count()

In [ ]:
#https://spark.apache.org/docs/2.1.0/ml-classification-regression.html#random-forest-classifier

from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationMetrics


rf = RandomForestClassifier(labelCol="cache", featuresCol="features", numTrees=10)
model = rf.fit(train_data)
predictions = model.transform(test_data)
predictions.select("prediction", "label", "features").show(n=5)



In [ ]:
# https://spark.apache.org/docs/1.5.1/mllib-evaluation-metrics.html#binary-classification
predictionAndLabels = test_data.map(lambda lp: (float(model.predict(lp.features)), lp.cache))
